# Restituate an explainability with a SmartPredictor in production

Shapash create a SmartPredictor to make prediction and have explainability in production context <br />
Explainability can be restitutate to users to have a simple synthetic explaination <br />

Contents:
- Build a SmartPredictor
- Save and Load a Smartpredictor
- Add input
- Use label and wording
- Summarize explaination

We used Kaggle's [Titanic](https://www.kaggle.com/c/titanic) dataset

In [1]:
import numpy as np
import pandas as pd
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import shap

In [3]:
from shapash.data.data_loader import data_loading

In [4]:
titan_df, titan_dict = data_loading('titanic')
del titan_df['Name']

In [5]:
titan_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,,
1,0,Third class,male,22.0,1,0,7.25,Southampton,Mr
2,1,First class,female,38.0,1,0,71.28,Cherbourg,Mrs
3,1,Third class,female,26.0,0,0,7.92,Southampton,Miss
4,1,First class,female,35.0,1,0,53.10,Southampton,Mrs
5,0,Third class,male,35.0,0,0,8.05,Southampton,Mr


## Create Classification Model

In [6]:
y = titan_df['Survived']
X = titan_df.drop('Survived', axis=1)

In [7]:
varcat=['Pclass','Sex','Embarked','Title']

In [8]:
categ_encoding = OrdinalEncoder(cols=varcat, \
                                handle_unknown='ignore', \
                                return_df=True).fit(X)
X = categ_encoding.transform(X)

/home/78257d/.conda/envs/test_env_shapash/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



Train Test split + Random Forest fit

In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.75, random_state=1)

rf = RandomForestClassifier(n_estimators=100,min_samples_leaf=3)
rf.fit(Xtrain, ytrain)

RandomForestClassifier(min_samples_leaf=3)

In [10]:
ypred=pd.DataFrame(rf.predict(Xtest),columns=['pred'],index=Xtest.index)

# Create a Smarpredictor

In [12]:
from shapash.explainer.smart_explainer import SmartExplainer

SmartPredictor takes only necessary dicts of the model features

In [25]:
model_features_dict = {'Pclass': 'Ticket class',
 'Sex': 'Sex',
 'Age': 'Age',
 'SibSp': 'Relatives such as brother or wife',
 'Parch': 'Relatives like children or parents',
 'Fare': 'Passenger fare',
 'Embarked': 'Port of embarkation',
 'Title': 'Title of passenger'}

In [26]:
xpl = SmartExplainer(features_dict=model_features_dict)

In [27]:
xpl.compile(
    x=Xtest,
    model=rf,
    preprocessing=categ_encoding,
    y_pred=ypred
)

Backend: Shap TreeExplainer


In [29]:
predictor = xpl.to_smartpredictor()

#### Save your predictor in Pickle File

In [30]:
predictor.save('./predictor.pkl')

#### Load your predictor in Pickle File

In [31]:
predictor_load = load_smartpredictor('./predictor.pkl')

## Make a prediction with your Predictor

#### Add data

In [37]:
person_x = {'Pclass': 'Third class',
 'Sex': 'female',
 'Age': 36,
 'SibSp': 1,
 'Parch': 0,
 'Fare': 7.25,
 'Embarked': 'Cherbourg',
 'Title': 'Miss'}

In [38]:
predictor_load.add_input(x=person_x)

#### Make prediction

In [40]:
prediction = predictor_load.predict()

In [41]:
prediction.head()

,ypred
0,0


#### Get detailed explanability associated to the prediction

In [43]:
detailed_contributions = predictor_load.detail_contributions()

In [44]:
detailed_contributions.head()

,ypred,proba,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,0.516996,0.107138,-0.126514,0.0538406,-0.00202188,0.00360771,0.0702406,-0.0512124,-0.166944


#### Summarize explanability of the predictions

In [47]:
predictor_load.modify_mask(max_contrib=3)

In [48]:
explanation = predictor_load.summarize()

In [49]:
explanation.head()

,ypred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
0,0,Title of passenger,3,-0.166944,Sex,2,-0.126514,Ticket class,1,0.107138
